In [8]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

#Load Dataset
df=pd.read_csv(r'C:\Users\ASUS PC\Desktop\AMDARI INTERNSHIP\optiSecure\OptiSecure_RS\dataCleaning\df.csv', 
parse_dates=['IssuanceDate', 'ExpiryDate', 'ClaimDate', 'InteractionDate', 'ResponseTime', 'RenewalDate'])



In [2]:
#quick overview of dataset
print(df.head(5))



   CustomerID   Age Gender   Income       Location MaritalStatus  PolicyID  \
0           1  34.0   Male  56462.5    Port Krista      Divorced         1   
1           1  34.0   Male  56462.5    Port Krista      Divorced         1   
2           1  34.0   Male  56462.5    Port Krista      Divorced         1   
3           1  34.0   Male  56462.5    Port Krista      Divorced         1   
4           2  31.0    NaN  32839.4  South Lisaton           NaN         2   

  PolicyType  Premium                                    CoverageDetails  \
0     Health  561.731  Measure yard thought room start himself. Why t...   
1     Health  561.731  Measure yard thought room start himself. Why t...   
2     Health  561.731  Measure yard thought room start himself. Why t...   
3     Health  561.731  Measure yard thought room start himself. Why t...   
4     Health  148.739  Maintain certain management tonight. Tonight n...   

  IssuanceDate ExpiryDate  ClaimAmount ClaimType  ClaimDate InteractionTyp

In [3]:
#structure of dataset
print(f'Rows:{df.shape[0]}, Columns: {df.shape[1]}')



Rows:8051, Columns: 21


In [4]:
print(df.dtypes)

CustomerID                  int64
Age                       float64
Gender                     object
Income                    float64
Location                   object
MaritalStatus              object
PolicyID                    int64
PolicyType                 object
Premium                   float64
CoverageDetails            object
IssuanceDate       datetime64[ns]
ExpiryDate         datetime64[ns]
ClaimAmount               float64
ClaimType                  object
ClaimDate          datetime64[ns]
InteractionType            object
InteractionDate    datetime64[ns]
ResponseTime               object
RenewalStatus              object
RenewalDate        datetime64[ns]
RenewalPremium            float64
dtype: object


In [5]:
#Summary of dataset
print(df.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8051 entries, 0 to 8050
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CustomerID       8051 non-null   int64         
 1   Age              4142 non-null   float64       
 2   Gender           5533 non-null   object        
 3   Income           4191 non-null   float64       
 4   Location         7251 non-null   object        
 5   MaritalStatus    6021 non-null   object        
 6   PolicyID         8051 non-null   int64         
 7   PolicyType       8051 non-null   object        
 8   Premium          8051 non-null   float64       
 9   CoverageDetails  8051 non-null   object        
 10  IssuanceDate     8051 non-null   datetime64[ns]
 11  ExpiryDate       8051 non-null   datetime64[ns]
 12  ClaimAmount      7017 non-null   float64       
 13  ClaimType        7017 non-null   object        
 14  ClaimDate        7017 non-null   datetim

In [6]:
#quick stats of dataset
print(df.describe(exclude='object'))



        CustomerID          Age         Income     PolicyID      Premium  \
count  8051.000000  4142.000000    4191.000000  8051.000000  8051.000000   
mean    751.309775    50.505311   85943.620329   751.309775   557.788515   
min       1.000000    18.000000   20104.300000     1.000000   100.164000   
25%     373.000000    37.000000   56628.800000   373.000000   324.231000   
50%     744.000000    51.000000   84365.000000   744.000000   572.883000   
75%    1114.000000    65.000000  117068.000000  1114.000000   789.233000   
max    1500.000000    80.000000  149453.000000  1500.000000   999.522000   
std     430.125367    17.415123   36397.662530   430.125367   264.653030   

                        IssuanceDate                     ExpiryDate  \
count                           8051                           8051   
mean   2022-10-23 10:19:55.677555456  2023-01-23 05:40:00.745249024   
min              2020-01-02 00:00:00            2020-01-03 00:00:00   
25%              2021-04-21 00:

In [7]:
#handle missing values
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values>0].sort_values(ascending=False)
print(missing_values)



Age                3909
Income             3860
Gender             2518
RenewalStatus      2050
MaritalStatus      2030
InteractionType    1978
ClaimDate          1034
ClaimAmount        1034
ClaimType          1034
ResponseTime        844
Location            800
RenewalPremium      762
dtype: int64


In [10]:
#check for duplicated values
print(df.duplicated().any())

False


In [24]:
num_col = df.select_dtypes(include=[np.number]).columns.tolist()
num_col

['CustomerID',
 'Age',
 'Income',
 'PolicyID',
 'Premium',
 'ClaimAmount',
 'ResponseTime',
 'RenewalPremium']

In [25]:
cat_col = df.select_dtypes(include=[np.object_]).columns.tolist()
cat_col

['Gender',
 'Location',
 'MaritalStatus',
 'PolicyType',
 'CoverageDetails',
 'IssuanceDate',
 'ExpiryDate',
 'ClaimType',
 'ClaimDate',
 'InteractionType',
 'InteractionDate',
 'RenewalStatus',
 'RenewalDate']

In [21]:
cat_col = df.select_dtypes(include=["object"]).columns.tolist()

for col in cat_col:
    df[col] = df[col].astype('category')
    df[col] = df[col].str.strip().str.lower()

df




,CustomerID,Age,Gender,Income,Location,MaritalStatus,PolicyID,PolicyType,Premium,CoverageDetails,IssuanceDate,ExpiryDate,ClaimAmount,ClaimType,ClaimDate,InteractionType,InteractionDate,ResponseTime,RenewalStatus,RenewalDate,RenewalPremium
0,1,34.0,male,56462.5,port krista,divorced,1,health,561.731,measure yard thought room start himself. why t...,2023-12-05,2021-04-25,1048.770,health,2025-05-23,website,2025-01-05,NaN,not renewed,2025-03-11,738.443
1,1,34.0,male,56462.5,port krista,divorced,1,health,561.731,measure yard thought room start himself. why t...,2023-12-05,2021-04-25,1048.770,health,2025-05-23,call center,2025-07-10,NaN,not renewed,2025-03-11,738.443
2,1,34.0,male,56462.5,port krista,divorced,1,health,561.731,measure yard thought room start himself. why t...,2023-12-05,2021-04-25,539.576,health,2025-02-07,website,2025-01-05,NaN,not renewed,2025-03-11,738.443
3,1,34.0,male,56462.5,port krista,divorced,1,health,561.731,measure yard thought room start himself. why t...,2023-12-05,2021-04-25,539.576,health,2025-02-07,call center,2025-07-10,NaN,not renewed,2025-03-11,738.443
4,2,31.0,NaN,32839.4,south lisaton,NaN,2,health,148.739,maintain certain management tonight. tonight n...,2021-08-21,2020-01-12,965.311,accident,2025-05-13,NaN,2025-07-07,NaN,renewed,2025-01-24,794.892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8046,1499,NaN,NaN,120113.0,port greggborough,NaN,1499,health,789.233,financial degree science continue few onto. po...,2025-03-23,2022-06-19,1359.400,health,2025-07-25,call center,2025-01-28,50.0,pending,2025-02-22,234.516
8047,1499,NaN,NaN,120113.0,port greggborough,NaN,1499,health,789.233,financial degree science continue few onto. po...,2025-03-23,2022-06-19,1359.400,health,2025-07-25,NaN,2025-06-16,47.0,pending,2025-02-22,234.516
8048,1499,NaN,NaN,120113.0,port greggborough,NaN,1499,health,789.233,financial degree science continue few onto. po...,2025-03-23,2022-06-19,1359.400,health,2025-07-25,mobile app,2025-04-18,50.0,pending,2025-02-22,234.516
8049,1500,33.0,NaN,NaN,new colleen,divorced,1500,health,681.518,enjoy admit west specific affect. radio sing r...,2020-02-21,2024-09-20,1113.580,accident,2025-04-05,website,2025-05-29,50.0,NaN,2025-06-29,311.630


In [ ]:
#Detect and handle outliers in num_col using IQR

num_col = df.select_dtypes(include=[np.number]).columns.tolist()

for col in num_col:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)
    
    df[col] = np.where((df[col] < lower_bound) | (df[col] > upper_bound), 0, df[col])

df


,CustomerID,Age,Gender,Income,Location,MaritalStatus,PolicyID,PolicyType,Premium,CoverageDetails,IssuanceDate,ExpiryDate,ClaimAmount,ClaimType,ClaimDate,InteractionType,InteractionDate,ResponseTime,RenewalStatus,RenewalDate,RenewalPremium
0,1,34.0,male,56462.5,port krista,divorced,1,health,561.731,measure yard thought room start himself. why t...,2023-12-05,2021-04-25,1048.770,health,2025-05-23,website,2025-01-05,NaN,not renewed,2025-03-11,738.443
1,1,34.0,male,56462.5,port krista,divorced,1,health,561.731,measure yard thought room start himself. why t...,2023-12-05,2021-04-25,1048.770,health,2025-05-23,call center,2025-07-10,NaN,not renewed,2025-03-11,738.443
2,1,34.0,male,56462.5,port krista,divorced,1,health,561.731,measure yard thought room start himself. why t...,2023-12-05,2021-04-25,539.576,health,2025-02-07,website,2025-01-05,NaN,not renewed,2025-03-11,738.443
3,1,34.0,male,56462.5,port krista,divorced,1,health,561.731,measure yard thought room start himself. why t...,2023-12-05,2021-04-25,539.576,health,2025-02-07,call center,2025-07-10,NaN,not renewed,2025-03-11,738.443
4,2,31.0,NaN,32839.4,south lisaton,NaN,2,health,148.739,maintain certain management tonight. tonight n...,2021-08-21,2020-01-12,965.311,accident,2025-05-13,NaN,2025-07-07,NaN,renewed,2025-01-24,794.892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8046,1499,NaN,NaN,120113.0,port greggborough,NaN,1499,health,789.233,financial degree science continue few onto. po...,2025-03-23,2022-06-19,1359.400,health,2025-07-25,call center,2025-01-28,50.0,pending,2025-02-22,234.516
8047,1499,NaN,NaN,120113.0,port greggborough,NaN,1499,health,789.233,financial degree science continue few onto. po...,2025-03-23,2022-06-19,1359.400,health,2025-07-25,NaN,2025-06-16,47.0,pending,2025-02-22,234.516
8048,1499,NaN,NaN,120113.0,port greggborough,NaN,1499,health,789.233,financial degree science continue few onto. po...,2025-03-23,2022-06-19,1359.400,health,2025-07-25,mobile app,2025-04-18,50.0,pending,2025-02-22,234.516
8049,1500,33.0,NaN,NaN,new colleen,divorced,1500,health,681.518,enjoy admit west specific affect. radio sing r...,2020-02-21,2024-09-20,1113.580,accident,2025-04-05,website,2025-05-29,50.0,NaN,2025-06-29,311.630


In [28]:
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()
df_scale = ['Income', 'Premium']

# Fit and transform
df[df_scale] = scale.fit_transform(df[df_scale])

df


,CustomerID,Age,Gender,Income,Location,MaritalStatus,PolicyID,PolicyType,Premium,CoverageDetails,IssuanceDate,ExpiryDate,ClaimAmount,ClaimType,ClaimDate,InteractionType,InteractionDate,ResponseTime,RenewalStatus,RenewalDate,RenewalPremium
0,1,34.0,male,-0.810070,port krista,divorced,1,health,0.014898,measure yard thought room start himself. why t...,2023-12-05,2021-04-25,1048.770,health,2025-05-23,website,2025-01-05,NaN,not renewed,2025-03-11,738.443
1,1,34.0,male,-0.810070,port krista,divorced,1,health,0.014898,measure yard thought room start himself. why t...,2023-12-05,2021-04-25,1048.770,health,2025-05-23,call center,2025-07-10,NaN,not renewed,2025-03-11,738.443
2,1,34.0,male,-0.810070,port krista,divorced,1,health,0.014898,measure yard thought room start himself. why t...,2023-12-05,2021-04-25,539.576,health,2025-02-07,website,2025-01-05,NaN,not renewed,2025-03-11,738.443
3,1,34.0,male,-0.810070,port krista,divorced,1,health,0.014898,measure yard thought room start himself. why t...,2023-12-05,2021-04-25,539.576,health,2025-02-07,call center,2025-07-10,NaN,not renewed,2025-03-11,738.443
4,2,31.0,NaN,-1.459175,south lisaton,NaN,2,health,-1.545703,maintain certain management tonight. tonight n...,2021-08-21,2020-01-12,965.311,accident,2025-05-13,NaN,2025-07-07,NaN,renewed,2025-01-24,794.892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8046,1499,NaN,NaN,0.938892,port greggborough,NaN,1499,health,0.874575,financial degree science continue few onto. po...,2025-03-23,2022-06-19,1359.400,health,2025-07-25,call center,2025-01-28,50.0,pending,2025-02-22,234.516
8047,1499,NaN,NaN,0.938892,port greggborough,NaN,1499,health,0.874575,financial degree science continue few onto. po...,2025-03-23,2022-06-19,1359.400,health,2025-07-25,NaN,2025-06-16,47.0,pending,2025-02-22,234.516
8048,1499,NaN,NaN,0.938892,port greggborough,NaN,1499,health,0.874575,financial degree science continue few onto. po...,2025-03-23,2022-06-19,1359.400,health,2025-07-25,mobile app,2025-04-18,50.0,pending,2025-02-22,234.516
8049,1500,33.0,NaN,NaN,new colleen,divorced,1500,health,0.467545,enjoy admit west specific affect. radio sing r...,2020-02-21,2024-09-20,1113.580,accident,2025-04-05,website,2025-05-29,50.0,NaN,2025-06-29,311.630


In [23]:
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8051 entries, 0 to 8050
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerID       8051 non-null   int64  
 1   Age              4142 non-null   float64
 2   Gender           5533 non-null   object 
 3   Income           4191 non-null   float64
 4   Location         7251 non-null   object 
 5   MaritalStatus    6021 non-null   object 
 6   PolicyID         8051 non-null   int64  
 7   PolicyType       8051 non-null   object 
 8   Premium          8051 non-null   float64
 9   CoverageDetails  8051 non-null   object 
 10  IssuanceDate     8051 non-null   object 
 11  ExpiryDate       8051 non-null   object 
 12  ClaimAmount      7017 non-null   float64
 13  ClaimType        7017 non-null   object 
 14  ClaimDate        7017 non-null   object 
 15  InteractionType  6073 non-null   object 
 16  InteractionDate  8051 non-null   object 
 17  ResponseTime  

,CustomerID,Age,Gender,Income,Location,MaritalStatus,PolicyID,PolicyType,Premium,CoverageDetails,IssuanceDate,ExpiryDate,ClaimAmount,ClaimType,ClaimDate,InteractionType,InteractionDate,ResponseTime,RenewalStatus,RenewalDate,RenewalPremium
0,1,34.0,male,56462.5,port krista,divorced,1,health,561.731,measure yard thought room start himself. why t...,2023-12-05,2021-04-25,1048.770,health,2025-05-23,website,2025-01-05,NaN,not renewed,2025-03-11,738.443
1,1,34.0,male,56462.5,port krista,divorced,1,health,561.731,measure yard thought room start himself. why t...,2023-12-05,2021-04-25,1048.770,health,2025-05-23,call center,2025-07-10,NaN,not renewed,2025-03-11,738.443
2,1,34.0,male,56462.5,port krista,divorced,1,health,561.731,measure yard thought room start himself. why t...,2023-12-05,2021-04-25,539.576,health,2025-02-07,website,2025-01-05,NaN,not renewed,2025-03-11,738.443
3,1,34.0,male,56462.5,port krista,divorced,1,health,561.731,measure yard thought room start himself. why t...,2023-12-05,2021-04-25,539.576,health,2025-02-07,call center,2025-07-10,NaN,not renewed,2025-03-11,738.443
4,2,31.0,NaN,32839.4,south lisaton,NaN,2,health,148.739,maintain certain management tonight. tonight n...,2021-08-21,2020-01-12,965.311,accident,2025-05-13,NaN,2025-07-07,NaN,renewed,2025-01-24,794.892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8046,1499,NaN,NaN,120113.0,port greggborough,NaN,1499,health,789.233,financial degree science continue few onto. po...,2025-03-23,2022-06-19,1359.400,health,2025-07-25,call center,2025-01-28,50.0,pending,2025-02-22,234.516
8047,1499,NaN,NaN,120113.0,port greggborough,NaN,1499,health,789.233,financial degree science continue few onto. po...,2025-03-23,2022-06-19,1359.400,health,2025-07-25,NaN,2025-06-16,47.0,pending,2025-02-22,234.516
8048,1499,NaN,NaN,120113.0,port greggborough,NaN,1499,health,789.233,financial degree science continue few onto. po...,2025-03-23,2022-06-19,1359.400,health,2025-07-25,mobile app,2025-04-18,50.0,pending,2025-02-22,234.516
8049,1500,33.0,NaN,NaN,new colleen,divorced,1500,health,681.518,enjoy admit west specific affect. radio sing r...,2020-02-21,2024-09-20,1113.580,accident,2025-04-05,website,2025-05-29,50.0,NaN,2025-06-29,311.630
